### Esta sección muestra la visualización de datos tabulares utilizando la clase Styler.

## Objeto Styler y HTML

El estilo debe realizarse después de que se hayan procesado los datos en un DataFrame. Styler crea una <tabla> HTML y aprovecha el lenguaje de estilo CSS para manipular muchos parámetros, incluidos colores, fuentes, bordes, fondo, etc.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl

In [2]:
df = pd.DataFrame([[38.0, 2.0, 18.0, 22.0, 21, np.nan],[19, 439, 6, 452, 226,232]],
                  index=pd.Index(['Tumour (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'),
                  columns=pd.MultiIndex.from_product([['Decision Tree', 'Regression', 'Random'],['Tumour', 'Non-Tumour']], names=['Model:', 'Predicted:']))

In [3]:
df.style

## Formato de salida

### Formato de valores

Antes de agregar estilos, es útil mostrar que Styler puede distinguir el valor de visualización del valor real, tanto en valores de datos como en encabezados de columnas o índices. Para controlar el valor de visualización, el texto se imprime en cada celda como una cadena, y podemos usar los métodos .format() y .format_index() para manipular esto de acuerdo con una cadena de especificación de formato o un invocable que toma un valor único y devuelve una cadena.

In [5]:
df.style.format(precision=0, na_rep='MISSING', thousands=" ",
                formatter={('Decision Tree', 'Tumour'): "{:.2f}",
                           ('Regression', 'Non-Tumour'): lambda x: "$ {:,.1f}".format(x*-1e6)
                          })

El uso de Styler para manipular la pantalla es una característica útil porque mantener la indexación y los valores de datos para otros fines brinda un mayor control. No es necesario que sobrescriba su DataFrame para mostrarlo como desee.

In [6]:
weather_df = pd.DataFrame(np.random.rand(10,2)*5,
                          index=pd.date_range(start="2021-01-01", periods=10),
                          columns=["Tokyo", "Beijing"])

In [7]:
weather_df

,Tokyo,Beijing
2021-01-01,3.301769,3.539691
2021-01-02,1.664808,2.821169
2021-01-03,0.100169,2.044158
2021-01-04,1.922374,0.209012
2021-01-05,0.966047,2.131665
2021-01-06,3.469721,2.668998
2021-01-07,4.868895,1.916475
2021-01-08,0.351810,4.886739
2021-01-09,1.997750,1.707947
2021-01-10,1.787138,4.621851


In [8]:
def rain_condition(v):
    if v < 1.75:
        return "Dry"
    elif v < 2.75:
        return "Rain"
    return "Heavy Rain"

In [9]:
def make_pretty(styler):
    styler.set_caption("Weather Conditions")
    styler.format(rain_condition)
    styler.format_index(lambda v: v.strftime("%A"))
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    return styler

In [10]:
weather_df

,Tokyo,Beijing
2021-01-01,3.301769,3.539691
2021-01-02,1.664808,2.821169
2021-01-03,0.100169,2.044158
2021-01-04,1.922374,0.209012
2021-01-05,0.966047,2.131665
2021-01-06,3.469721,2.668998
2021-01-07,4.868895,1.916475
2021-01-08,0.351810,4.886739
2021-01-09,1.997750,1.707947
2021-01-10,1.787138,4.621851


In [11]:
weather_df.loc["2021-01-04":"2021-01-08"].style.pipe(make_pretty)

,Tokyo,Beijing
Monday,Rain,Dry
Tuesday,Dry,Rain
Wednesday,Heavy Rain,Rain
Thursday,Heavy Rain,Rain
Friday,Dry,Heavy Rain


In [12]:
s = df.style.format('{:.0f}').hide([('Random', 'Tumour'), ('Random', 'Non-Tumour')], axis="columns")
s

In [15]:
 cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000066; color: white;'
}
s.set_table_styles([cell_hover, index_names, headers])

In [16]:
s.set_table_styles([
    {'selector': 'th.col_heading', 'props': 'text-align: center;'},
    {'selector': 'th.col_heading.level0', 'props': 'font-size: 1.5em;'},
    {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
], overwrite=False)

In [17]:
# Podemos establecer estilos utilizando un diccionario
s.set_table_styles({
    ('Regression', 'Tumour'): [{'selector': 'th', 'props': 'border-left: 1px solid white'},
                               {'selector': 'td', 'props': 'border-left: 1px solid #000066'}]
}, overwrite=False, axis=0)

## Funciones apply() y applymap()

In [18]:
np.random.seed(0)
df2 = pd.DataFrame(np.random.randn(10,4), columns=['A','B','C','D'])
df2.style

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [19]:
def style_negative(v, props=''):
    return props if v < 0 else None
s2 = df2.style.applymap(style_negative, props='color:red;')\
              .applymap(lambda v: 'opacity: 20%;' if (v < 0.3) and (v > -0.3) else None)
s2

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [20]:
def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')
s2.apply(highlight_max, props='color:white;background-color:darkblue', axis=0)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [21]:
s2.apply(highlight_max, props='color:white;background-color:pink;', axis=1)\
  .apply(highlight_max, props='color:white;background-color:purple', axis=None)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


## Funciones que actuán sobre los índices .applymap_index() y .apply_index() 

In [22]:
s2.applymap_index(lambda v: "color:pink;" if v>4 else "color:darkblue;", axis=0)
s2.apply_index(lambda s: np.where(s.isin(["A", "B"]), "color:pink;", "color:darkblue;"), axis=1)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


## Información sobre herramientas y subtítulos -  .set_caption()

In [23]:
s.set_caption("Confusion matrix for multiple cancer prediction models.")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: bottom; font-size:1.25em;'
 }], overwrite=False)

In [24]:
tt = pd.DataFrame([['This model has a very strong true positive rate',
                    "This model's total number of false negatives is too high"]],
                  index=['Tumour (Positive)'], columns=df.columns[[0,3]])
s.set_tooltips(tt, props='visibility: hidden; position: absolute; z-index: 1; border: 1px solid #000066;'
                         'background-color: white; color: #000066; font-size: 0.8em;'
                         'transform: translate(0px, -24px); padding: 0.6em; border-radius: 0.5em;')

In [1]:
# s.set_table_styles([  # create internal CSS classes
#     {'selector': '.border-red', 'props': 'border: 2px dashed red;'},
#     {'selector': '.border-green', 'props': 'border: 2px dashed green;'},
# ], overwrite=False)
# cell_border = pd.DataFrame([['border-green ', ' ', ' ', 'border-red '],
#                            [' ', ' ', ' ', ' ']],
#                           index=df.index,
#                           columns=df.columns[:4])
# s.set_td_classes(cell_color + cell_border)

## Control específico del slicing

In [26]:
df3 = pd.DataFrame(np.random.randn(4,4),
                   pd.MultiIndex.from_product([['A', 'B'], ['r1', 'r2']]),
                   columns=['c1','c2','c3','c4'])
df3

c1        c2        c3        c4
A r1 -1.048553 -1.420018 -1.706270  1.950775
  r2 -0.509652 -0.438074 -1.252795  0.777490
B r1 -1.613898 -0.212740 -0.895467  0.386902
  r2 -0.510805 -1.180632 -0.028182  0.428332

In [27]:
slice_ = ['c3', 'c4']
df3.style.apply(highlight_max, props='color:red;', axis=0, subset=slice_)\
         .set_properties(**{'background-color': '#ffffb3'}, subset=slice_)

In [28]:
idx = pd.IndexSlice
slice_ = idx[idx[:,'r1'], idx['c2':'c4']]
df3.style.apply(highlight_max, props='color:red;', axis=0, subset=slice_)\
         .set_properties(**{'background-color': '#ffffb3'}, subset=slice_)

In [29]:
slice_ = idx[idx[:,'r2'], :]
df3.style.apply(highlight_max, props='color:red;', axis=1, subset=slice_)\
         .set_properties(**{'background-color': '#ffffb3'}, subset=slice_)

In [30]:
slice_ = idx[idx[(df3['c1'] + df3['c3']) < -2.0], ['c2', 'c4']]
df3.style.apply(highlight_max, props='color:red;', axis=1, subset=slice_)\
         .set_properties(**{'background-color': '#ffffb3'}, subset=slice_)

## Estilos predeterminados (no requieren funciones)

In [31]:
# .highlight_null: for use with identifying missing data.
# .highlight_min and .highlight_max: for use with identifying extremeties in data.
# .highlight_between and .highlight_quantile: for use with identifying classes within data.
# .background_gradient: a flexible method for highlighting cells based on their, or other, values on a numeric scale.
# .text_gradient: similar method for highlighting text based on their, or other, values on a numeric scale.
# .bar: to display mini-charts within cell backgrounds.

### 1. Resaltar valores nulos - .highlight_null

In [34]:
df2.iloc[0,2] = np.nan
df2.iloc[4,3] = np.nan
df2.loc[:4].style.highlight_null("yellow")

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan


### 2. Resaltar mínimos y máximos - .highlight_min and .highlight_max

In [35]:
df2.loc[:4].style.highlight_max(axis=1, props='color:white; font-weight:bold; background-color:darkblue;')

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan


### 3. Resaltar rangos y cuantiles - .highlight_between and .highlight_quantile 

In [36]:
left = pd.Series([1.0, 0.0, 1.0], index=["A", "B", "D"])
df2.loc[:4].style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:purple;')

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan


In [37]:
# Cuantiles
df2.loc[:4].style.highlight_quantile(q_left=0.85, axis=None, color='yellow')

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan


### 4. Degradado de fondo y degradado de texto

In [38]:
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)

df2.style.background_gradient(cmap=cm)

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [39]:
df2.style.text_gradient(cmap=cm)

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


### 5. Definir propiedades de forma sencilla - .style.set_properties

In [40]:
df2.loc[:4].style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen',
                           'border-color': 'white'})

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan


### 6. Gráficos de barra dentro de la tabla

In [41]:
df2.style.bar(subset=['A', 'B'], color='#d65f5f')

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [42]:
df2.style.format('{:.3f}', na_rep="")\
         .bar(align=0, vmin=-2.5, vmax=2.5, cmap="bwr", height=50,
              width=60, props="width: 120px; border-right: 1px solid black;")\
         .text_gradient(cmap="bwr", vmin=-2.5, vmax=2.5)

,A,B,C,D
0,1.764,0.400,,2.241
1,1.868,-0.977,0.950,-0.151
2,-0.103,0.411,0.144,1.454
3,0.761,0.122,0.444,0.334
4,1.494,-0.205,0.313,
5,-2.553,0.654,0.864,-0.742
6,2.270,-1.454,0.046,-0.187
7,1.533,1.469,0.155,0.378
8,-0.888,-1.981,-0.348,0.156
9,1.230,1.202,-0.387,-0.302


### 7.  Encabezados fijos

In [44]:
bigdf = pd.DataFrame(np.random.randn(16, 100))
bigdf.style.set_sticky(axis="index")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.066517,0.302472,-0.634322,-0.362741,-0.672460,-0.359553,-0.813146,-1.726283,0.177426,-0.401781,-1.630198,0.462782,-0.907298,0.051945,0.729091,0.128983,1.139401,-1.234826,0.402342,-0.684810,-0.870797,-0.578850,-0.311553,0.056165,-1.165150,0.900826,0.465662,-1.536244,1.488252,1.895889,1.178780,-0.179925,-1.070753,1.054452,-0.403177,1.222445,0.208275,0.976639,0.356366,0.706573,0.010500,1.785870,0.126912,0.401989,1.883151,-1.347759,-1.270485,0.969397,-1.173123,1.943621,-0.413619,-0.747455,1.922942,1.480515,1.867559,0.906045,-0.861226,1.910065,-0.268003,0.802456,0.947252,-0.155010,0.614079,0.922207,0.376426,-1.099401,0.298238,1.326386,-0.694568,-0.149635,-0.435154,1.849264,0.672295,0.407462,-0.769916,0.539249,-0.674333,0.031831,-0.635846,0.676433,0.576591,-0.208299,0.396007,-1.093062,-1.491258,0.439392,0.166673,0.635031,2.383145,0.944479,-0.912822,1.117016,-1.315907,-0.461585,-0.068242,1.713343,-0.744755,-0.826439,-0.098453,-0.663478
1,1.126636,-1.079932,-1.147469,-0.437820,-0.498032,1.929532,0.949421,0.087551,-1.225436,0.844363,-1.000215,-1.544771,1.188030,0.316943,0.920859,0.318728,0.856831,-0.651026,-1.034243,0.681595,-0.803410,-0.689550,-0.455533,0.017479,-0.353994,-1.374951,-0.643618,-2.223403,0.625231,-1.602058,-1.104383,0.052165,-0.739563,1.543015,-1.292857,0.267051,-0.039283,-1.168093,0.523277,-0.171546,0.771791,0.823504,2.163236,1.336528,-0.369182,-0.239379,1.099660,0.655264,0.640132,-1.616956,-0.024326,-0.738031,0.279925,-0.098150,0.910179,0.317218,0.786328,-0.466419,-0.944446,-0.410050,-0.017020,0.379152,2.259309,-0.042257,-0.955945,-0.345982,-0.463596,0.481481,-1.540797,0.063262,0.156507,0.232181,-0.597316,-0.237922,-1.424061,-0.493320,-0.542861,0.416050,-1.156182,0.781198,1.494485,-2.069985,0.426259,0.676908,-0.637437,-0.397272,-0.132881,-0.297791,-0.309013,-1.676004,1.152332,1.079619,-0.813364,-1.466424,0.521065,-0.575788,0.141953,-0.319328,0.691539,0.694749
2,-0.725597,-1.383364,-1.582938,0.610379,-1.188859,-0.506816,-0.596314,-0.052567,-1.936280,0.188779,0.523891,0.088422,-0.310886,0.097400,0.399046,-2.772593,1.955912,0.390093,-0.652409,-0.390953,0.493742,-0.116104,-2.030684,2.064493,-0.110541,1.020173,-0.692050,1.536377,0.286344,0.608844,-1.045253,1.211145,0.689818,1.301846,-0.628088,-0.481027,2.303917,-1.060016,-0.135950,1.136891,0.097725,0.582954,-0.399449,0.370056,-1.306527,1.658131,-0.118164,-0.680178,0.666383,-0.460720,-1.334258,-1.346718,0.693773,-0.159573,-0.133702,1.077744,-1.126826,-0.730678,-0.384880,0.094352,-0.042171,-0.286887,-0.061626,-0.107305,-0.719604,-0.812993,0.274516,-0.890915,-1.157355,-0.312292,-0.157667,2.256723,-0.704700,0.943261,0.747188,-1.188945,0.773253,-1.183881,-2.659172,0.606320,-1.755891,0.450934,-0.684011,1.659551,1.068509,-0.453386,-0.687838,-1.214077,-0.440923,-0.280355,-0.364694,0.156704,0.578521,0.349654,-0.764144,-1.437791,1.364532,-0.689449,-0.652294,-0.521189
3,-1.843070,-0.477974,-0.479656,0.620358,0.698457,0.003771,0.931848,0.339965,-0.015682,0.160928,-0.190653,-0.394850,-0.267734,-1.128011,0.280442,-0.993124,0.841631,-0.249459,0.049495,0.493837,0.643314,-1.570623,-0.206904,0.880179,-1.698106,0.387280,-2.255564,-1.022507,0.038631,-1.656715,-0.985511,-1.471835,1.648135,0.164228,0.567290,-0.222675,-0.353432,-1.616474,-0.291837,-0.761492,0.857924,1.141102,1.466579,0.852552,-0.598654,-1.115897,0.766663,0.356293,-1.768538,0.355482,0.814520,0.058926,-0.185054,-0.807648,-1.446535,0.800298,-0.309114,-0.233467,1.732721,0.684501,0.370825,0.142062,1.519995,1.719589,0.929505,0.582225,-2.094603,0.123722,-0.130107,0.093953,0.943046,-2.739677,-0.569312,0.269904,-0.466846,-1.416906,0.868963,0.276872,-0.971105,0.314817,0.821586,0.005293,0.800565,0.078260,-0.395229,-1.159421,-0.085931,0.194293,0.875833

In [45]:
bigdf.index = pd.MultiIndex.from_product([["A","B"],[0,1],[0,1,2,3]])
bigdf.style.set_sticky(axis="index", pixel_size=18, levels=[1,2])

### 8. Exportar a Excel

In [46]:
df2.style.\
    applymap(style_negative, props='color:red;').\
    highlight_max(axis=0).\
    to_excel('styled.xlsx', engine='openpyxl')